In [92]:
import pandas as pd
import torch
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pprint import pprint as print
import seaborn as sns
plt.style.use('seaborn-white')
import warnings
warnings.filterwarnings('ignore')
from navec import Navec
from slovnet.model.emb import NavecEmbedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from loguru import logger
import nltk, sys
from stop_words import get_stop_words
from pymystem3 import Mystem
from string import punctuation
from loguru import logger
import tensorflow_hub as hub
import tensorflow_models as tfm
import os
import tensorflow_text as text
sns.set_palette("pastel")

In [34]:
!gdown 1boACz8ab9UytCpi0QhTUFRyS2oyH1MXS

Downloading...
From: https://drive.google.com/uc?id=1boACz8ab9UytCpi0QhTUFRyS2oyH1MXS
To: /Users/glebsokolov/HeadRepo/test_data.csv
100%|██████████████████████████████████████| 53.4k/53.4k [00:00<00:00, 1.04MB/s]


In [102]:
path = '/Users/glebsokolov/HeadRepo/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)
emb = NavecEmbedding(navec)
cosine_similarity = lambda x,y: np.dot(x, y)/(np.linalg.norm(x, 2)*np.linalg.norm(y, 2))

In [103]:
def cosine_similarity_calc(vec_1,vec_2):
	sim = np.dot(vec_1,vec_2)/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))
	return sim

In [5]:
NUM_WORDS = 1000
EMBEDDING_DIM = 300
MAXLEN = 120
PADDING = 'post'
OOV_TOKEN = "<OOV>"
TRAINING_SPLIT = .8

In [6]:
mystem = Mystem()
russian_stopwords = get_stop_words('ru')
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [129]:
path='/Users/glebsokolov/HeadRepo/test_data.csv'
df = pd.read_csv(path)
df.head()
print(f'The length of dataframe is {len(df)}')
print(f'There are {max(df.dlg_id)} dialogues in there.')

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


'The length of dataframe is 480'
'There are 5 dialogues in there.'


In [130]:
tokenizer = Tokenizer(num_words = NUM_WORDS, oov_token=OOV_TOKEN)
texts_combined = [preprocess_text(i) for i in df.iloc[:,3]]
texts_cleared = [i for i in texts_combined if i !='']
print(f'The shape of the cleared dataframe is:{np.shape(texts_cleared)}')
tokenizer.fit_on_texts(texts_cleared)
word_index = tokenizer.word_index
print(f'Length of word index is {len(tokenizer.word_index)}')
sequence = lambda x: tokenizer.texts_to_sequences(x.split(' '))
sequences = pad_sequences([sequence(x) for x in texts_cleared], maxlen=MAXLEN, padding=PADDING)
print(f'The shape of padded sequences are {np.shape(sequences)}')
# keyword = lambda x: [i for i in word_index if word_index[i]==x]

'The shape of the cleared dataframe is:(424,)'
'Length of word index is 641'
'The shape of padded sequences are (424, 120, 1)'


In [96]:
input = torch.tensor([np.array(np.reshape(sequences[i],-1) )for i in range(len(texts_cleared))]).long()
embedded = tf.convert_to_tensor(emb(input).numpy())
np.shape(embedded)

TensorShape([424, 120, 300])

In [117]:
replies = [embedded[i,:,:] for i in range(len(texts_cleared))]
def calculate_max_similarity(sentence1, sentence2=navec['здравствуйте']):
    # find cosine similarity for each pair of words in the sentence
    similarities = np.array([])
    for ind in range(len(sentence1)):
        similarities = np.append(similarities, cosine_similarity(sentence1[ind], sentence2))
    return np.max(similarities)
        

In [118]:
similarities = np.array([])
for reply in replies:
    similarities = np.append(similarities, calculate_max_similarity(reply))

In [ ]:
model = tf.keras.Sequential([
    # tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])